In [1]:
import pandas as pd
import numpy as np
import os
import scipy.stats as stats

import seaborn as sns
import missingno as msno

In [2]:
def show_full_data(data, row_size=None, column_size=None, col_width=-1):
    """Shows all rows and columns instead of showing only some part and hiding other parts for large data.
    """
    with pd.option_context('display.max_rows', row_size, 'display.max_columns', column_size, 'display.max_colwidth', col_width):
        display(data)

# Get All User IDs

In [3]:
def get_user_list(loc):
    # Collecting all student codes from activity folder (which represents all students)
    user_codes = []
    for x in sorted(os.listdir(loc + 'sensing/activity/')):
        # Chooses the string before "." and after "_"
        user_codes.append(x.split('.')[0].split('_')[1])
    return user_codes

# Data Read Functions

## Sensing Data

In [4]:
def get_activity(user, loc):
    activity = pd.read_csv(loc + 'sensing/activity/activity_' + user + '.csv')
    activity.columns = ['timestamp', 'activity_inference']
    # make timestamp unique and take the mode for different values of activity inference
    activity = activity.groupby("timestamp")['activity_inference'].apply(lambda x: x.mode()[0]).reset_index()
    activity.timestamp = pd.to_datetime(activity.timestamp, unit='s')
    activity = activity.set_index('timestamp')
    activity = activity.asfreq('s', method='bfill')
    return activity

In [5]:
def get_audio(user, loc):
    audio = pd.read_csv(loc + 'sensing/audio/audio_' + user + '.csv')
    audio.columns = ['timestamp', 'audio_inference']
    # make timestamp unique and take the mode for different values of audio inference
    audio = audio.groupby("timestamp")['audio_inference'].apply(lambda x: x.mode()[0]).reset_index()
    audio.timestamp = pd.to_datetime(audio.timestamp, unit='s')
    audio = audio.set_index('timestamp')
    audio = audio.asfreq('s', method='bfill')
    return audio

In [6]:
def get_conversation(user, loc):
    conversation = pd.read_csv(loc + 'sensing/conversation/conversation_' + user + '.csv')
    conversation.columns = ['start_timestamp', 'end_timestamp']
    conversation.start_timestamp = pd.to_datetime(conversation.start_timestamp, unit='s')
    conversation.end_timestamp = pd.to_datetime(conversation.end_timestamp, unit='s')
    return conversation

In [7]:
def get_bluetooth(user, loc):
    bluetooth = pd.read_csv(loc + 'sensing/bluetooth/bt_' + user + '.csv', index_col=False)
    bluetooth.time = pd.to_datetime(bluetooth.time, unit='s')
    return bluetooth

In [8]:
def get_wifi(user, loc):
    wifi = pd.read_csv(loc + 'sensing/wifi/wifi_' + user + '.csv', index_col=False)
    wifi.time = pd.to_datetime(wifi.time, unit='s')
    return wifi

In [9]:
def get_wifi_loc(user, loc):
    wifi_loc = pd.read_csv(loc + 'sensing/wifi_location/wifi_location_' + user + '.csv', index_col=False)
    wifi_loc.time = pd.to_datetime(wifi_loc.time, unit='s')
    return wifi_loc

In [10]:
def get_dark(user, loc):
    dark = pd.read_csv(loc + 'sensing/dark/dark_' + user + '.csv', index_col=False)
    dark.start = pd.to_datetime(dark.start, unit='s')
    dark.end = pd.to_datetime(dark.end, unit='s')
    return dark

In [11]:
def get_phone_charge(user, loc):
    phonecharge = pd.read_csv(loc + 'sensing/phonecharge/phonecharge_' + user + '.csv', index_col=False)
    phonecharge.start = pd.to_datetime(phonecharge.start, unit='s')
    phonecharge.end = pd.to_datetime(phonecharge.end, unit='s')
    return phonecharge

In [12]:
def get_phone_lock(user, loc):
    phonelock = pd.read_csv(loc + 'sensing/phonelock/phonelock_' + user + '.csv', index_col=False)
    phonelock.start = pd.to_datetime(phonelock.start, unit='s')
    phonelock.end = pd.to_datetime(phonelock.end, unit='s')
    return phonelock

## Not Sensing

In [203]:
def get_sms(user, loc):
    sms = pd.read_csv(loc + 'sms/sms_' + user + '.csv', index_col=False)
    sms = sms[['timestamp']]
    sms = sms.groupby('timestamp').count().reset_index()
    sms['timestamp'] = pd.to_datetime(sms.timestamp, unit='s')
    sms['sms'] = 1
    sms = sms.set_index('timestamp')
    return sms

In [202]:
def get_cal_log(user, loc):
    call_log = pd.read_csv(loc + 'call_log/call_log_' + user + '.csv', index_col=False)
    if 'CALLS_date' not in call_log.columns:
        call_log = call_log[['timestamp']]
        call_log = call_log.groupby('timestamp').count().reset_index()
        call_log['timestamp'] = pd.to_datetime(call_log.timestamp, unit='s')
        call_log['call_log'] = 1
        call_log = call_log.set_index('timestamp')
    else:
        call_log['timestamp'] = pd.to_datetime(call_log.timestamp, unit='s')
        call_log['CALLS_date'] = pd.to_datetime(call_log.CALLS_date, unit='ms')
        call_log = call_log[['timestamp', 'CALLS_date', 'CALLS_duration']]
        call_log = call_log.groupby(['timestamp', 'CALLS_date']).sum().reset_index()
        call_log['call_log'] = 1
    return call_log

# Data Merger Functions

## Sensing

In [13]:
# Activity is the main dataframe, all other data are merged on it.
# In this case, df = activity.
def merge_audio(df, audio):
    df = pd.merge(df, audio, left_index=True, right_index=True, how='outer')
    return df.reset_index()

In [14]:
def merge_conversation(df, conversation):
    # add conversation
    df['conversation'] = np.nan
    for i in range(conversation.shape[0]):
        start = conversation.iloc[i, 0]
        end = conversation.iloc[i, 1]
        df.loc[(df['timestamp'] >= start) & (df['timestamp'] <= end), 'conversation'] = 1
    return df

In [15]:
def merge_bluetooth(df, bluetooth):
    # add bluetooth
    bluetooth_new = pd.DataFrame()
    for time in bluetooth.time.unique():
        data = {'timestamp': time}
        item = bluetooth[bluetooth.time == time]
        data['total_devices_around'] = item.shape[0]
        data['total_nearer'] = item[(item.level >= -65) & (item.level <= 0)].shape[0]
        data['total_near'] = item[(item.level >= -80) & (item.level < -65)].shape[0]
        data['total_far'] = item[(item.level >= -90) & (item.level < -80)].shape[0]
        data['total_farther'] = item[(item.level >= -125) & (item.level < -90)].shape[0] # Normally -100 is max but for one anomaly.
        data['level_avg'] = round(item.level.mean())
        data['level_std'] = item.level.std()
        bluetooth_new = bluetooth_new.append(data, ignore_index=True)
    bluetooth_new.columns = ['bt_' + i for i in bluetooth_new.columns]
    df = pd.merge(df, bluetooth_new, left_on='timestamp', right_on='bt_timestamp', how='outer')
    df.drop(columns=['bt_timestamp'], inplace=True)
    return df

In [16]:
def merge_wifi(df, wifi):
    # add wifi
    wifi_new = pd.DataFrame()
    for time in wifi.time.unique():
        data = {'timestamp': time}
        item = wifi[wifi.time == time]
        data['total_devices_around'] = item.shape[0]
        data['total_nearer'] = item[item.level >= -60].shape[0]
        data['total_near'] = item[(item.level >= -80) & (item.level < -60)].shape[0]
        data['total_far'] = item[(item.level >= -100) & (item.level < -80)].shape[0]
        data['level_avg'] = round(item.level.mean())
        data['level_std'] = item.level.std()
        wifi_new = wifi_new.append(data, ignore_index=True)
    wifi_new.columns = ['wifi_' + i for i in wifi_new.columns]
    df = pd.merge(df, wifi_new, left_on='timestamp', right_on='wifi_timestamp', how='left')
    df.drop(columns=['wifi_timestamp'], inplace=True)
    return df

In [17]:
def merge_dark(df, dark):
    # add dark
    df['phone_in_dark'] = np.nan
    for i in range(dark.shape[0]):
        start = dark.iloc[i, 0]
        end = dark.iloc[i, 1]
        df.loc[(df['timestamp'] >= start) & (df['timestamp'] <= end), 'phone_in_dark'] = 1
    return df

In [18]:
def merge_phone_charge(df, phone_charge):
    # phone charge
    df['phone_charging'] = np.nan
    for i in range(phone_charge.shape[0]):
        start = phone_charge.iloc[i, 0]
        end = phone_charge.iloc[i, 1]
        df.loc[(df['timestamp'] >= start) & (df['timestamp'] <= end), 'phone_charging'] = 1
    return df

In [19]:
def merge_phone_lock(df, phone_lock):
    # phone locked
    df['phone_locked'] = np.nan
    for i in range(phone_lock.shape[0]):
        start = phone_lock.iloc[i, 0]
        end = phone_lock.iloc[i, 1]
        df.loc[(df['timestamp'] >= start) & (df['timestamp'] <= end), 'phone_locked'] = 1
    return df

## Not Sensing

In [237]:
def merge_sms(df, sms):
    df = pd.merge_asof(df, sms, left_index=True, right_index=True, tolerance=pd.Timedelta('10m'))
    return df

In [238]:
def merge_call_log(df, call_log):
    if 'CALLS_date' in call_log.columns:
        only_time = call_log[['timestamp', 'call_log']]
        only_time = only_time.set_index('timestamp')
        df = pd.merge_asof(df, only_time, left_index=True, right_index=True, tolerance=pd.Timedelta('10m'))
        call_dur = call_log[['CALLS_date', 'CALLS_duration']]
        call_dur.columns = ['CALLS_date', 'call_duration']
        call_dur = call_dur.set_index('CALLS_date')
        df = pd.merge_asof(df, call_dur, left_index=True, right_index=True, tolerance=pd.Timedelta('10m'))
        df.loc[df.call_duration.notnull(), 'call_log'] = 1
    else:
        df = pd.merge_asof(df, call_log, left_index=True, right_index=True, tolerance=pd.Timedelta('10m'))
    return df

## Use all mergers

In [20]:
def merge_all(activity, audio, conversation, bluetooth, wifi, dark, phone_charge, phone_lock, sms):
    df = merge_audio(activity, audio)
    print('audio merge completed.')
    df = merge_conversation(df, conversation)
    print('conversation merge completed.')
    df = merge_bluetooth(df, bluetooth)
    print('bluetooth merge completed.')
    df = merge_wifi(df, wifi)
    print('wifi merge completed.')
    df = merge_dark(df, dark)
    print('dark merge completed.')
    df = merge_phone_charge(df, phone_charge)
    print('phone_charge merge completed.')
    df = merge_phone_lock(df, phone_lock)
    print('phone_lock merge completed.')
    df = merge_sms(df, sms)
    print('sms merge completed.')
    df = merge_call_log(df, call_log)
    return df

# EMA Functions

In [21]:
def ema(user, typ, cols, loc):
    data = pd.read_json(loc + 'EMA/response/' + typ + '/' + typ + '_' + user + '.json')
    if 'null' in data.columns:
        data = data.drop(columns='null')
    if 'location' in data.columns:
        data = data.drop(columns='location')
    # Checks if the given columns exists in data, if exists drop them.
    dr = True 
    for j in cols:
        if j not in data.columns:
            dr = False
    if dr == True:
        data = data.dropna(subset=cols)
    return data

In [22]:
def label_generator(stress, mood2, resample_factor):
    stress['level'] = stress['level'].replace([1,2,3], 1)
    stress['level'] = stress['level'].replace([4,5], 0)
    stress.columns = ['STRESSED', 'resp_time']
    mood2 = mood2.replace([1, 3], 0)
    mood2 = mood2.replace([2], 1)
    mood2.columns = ['STRESSED', 'resp_time']
    labels = stress.append(mood2)
    labels = labels.sort_values(by='resp_time', ascending=True)
    labels = labels.set_index('resp_time').resample(resample_factor).max()
    labels = labels[labels.STRESSED.notnull()]
    return labels

In [23]:
def resample_aggregations(columns):
    # Function to assign aggregation method during resampling
    agg_dict = {}
    for i in columns:
        if ('conversation' in i) | ('phone' in i) | ('inference' in i):
            agg_dict[i] = np.sum
        elif 'level' in i:
            agg_dict[i] = np.mean
        elif 'total' in i:
            agg_dict[i] = np.max
    return agg_dict

# MAIN

In [24]:
# Set dataset directory
dir_loc = '../../student-life-study-data/dataset/'

In [174]:
# Get user list
user_codes = get_user_list(dir_loc)

In [26]:
### REMOVE AFTER TEST
user_codes = user_codes[:3]

In [27]:
# Create empty dataset
dataset = pd.DataFrame()

In [28]:
### REMOVE
user = user_codes[0]

In [29]:
# Sensing
activity = get_activity(user, dir_loc)
print('activity data read for', user, 'is completed.')
audio = get_audio(user, dir_loc)
print('audio data read for', user, 'is completed.')
conversation = get_conversation(user, dir_loc)
print('conversation data read for', user, 'is completed.')
bluetooth = get_bluetooth(user, dir_loc)
print('bluetooth data read for', user, 'is completed.')
wifi = get_wifi(user, dir_loc)
print('wifi data read for', user, 'is completed.')
dark = get_dark(user, dir_loc)
print('dark data read for', user, 'is completed.')
phone_charge = get_phone_charge(user, dir_loc)
print('phone_charge data read for', user, 'is completed.')
phone_lock = get_phone_lock(user, dir_loc)
print('phone_lock data read for', user, 'is completed.')

# Not Sensing
sms = get_sms(user, dir_loc)
print('sms data read for', user, 'is completed.')
call_log = get_call_log(user, dir_loc)
print('call_log data read for', user, 'is completed.')

# EMA
stress = ema(user, 'Stress', ['level'], dir_loc)
print('stress data read for', user, 'is completed.')
mood2 = ema(user, 'Mood 2', ['how'], dir_loc)
print('mood2 data read for', user, 'is completed.')

activity data read for u00 is completed.
audio data read for u00 is completed.
conversation data read for u00 is completed.
bluetooth data read for u00 is completed.
wifi data read for u00 is completed.
dark data read for u00 is completed.
phone_charge data read for u00 is completed.
phone_lock data read for u00 is completed.
stress data read for u00 is completed.
mood2 data read for u00 is completed.


In [30]:
df = merge_all(activity, audio, conversation,
                bluetooth, wifi, dark,
                phone_charge, phone_lock,
               sms, call_log)

audio merge completed.
conversation merge completed.
bluetooth merge completed.
wifi merge completed.
dark merge completed.
phone_charge merge completed.
phone_lock merge completed.


In [32]:
# Create labels
if stress.shape[1] == 2:
    stress['level'] = stress['level'].replace([1,2,3], 1)
    stress['level'] = stress['level'].replace([4,5], 0)
    stress.columns = ['STRESSED', 'resp_time']

if mood2.shape[1] == 2:
    mood2 = mood2.replace([1, 3], 0)
    mood2 = mood2.replace([2], 1)
    mood2.columns = ['STRESSED', 'resp_time']

labels = stress.append(mood2)
labels = labels.sort_values(by='resp_time', ascending=True)
labels = labels.set_index('resp_time')

In [37]:
# Choose only valid timestamps
df_backup = df[df.timestamp.notnull()].copy()

In [38]:
df = df[df.timestamp.notnull()]

In [39]:
# Fill empty values in dataset.
df.loc[:, ['activity_inference',
           'audio_inference']] = df.loc[:, ['activity_inference',
                                             'audio_inference']].fillna(value=3)

In [40]:
df.loc[:, ['conversation',
            'phone_in_dark',
            'phone_charging',
            'phone_locked',
            'sms',
            'call_log']] = df.loc[:, ['conversation',
                                        'phone_in_dark',
                                        'phone_charging',
                                        'phone_locked',
                                        'sms',
                                        'call_log']].fillna(value=0)

In [41]:
df.loc[:, ['activity_inference', 
           'audio_inference']] = df.loc[:, ['activity_inference', 
                                            'audio_inference']].astype(int)

In [43]:
# One Hot Encode
df['activity_inference'] = df['activity_inference'].astype('category')
df['audio_inference'] = df['audio_inference'].astype('category')
df = pd.get_dummies(df)

In [44]:
# Resampling
df = df.set_index('timestamp')
df.index = pd.to_datetime(df.index, unit='s')

res_aggs = resample_aggregations(list(df.columns))
df = df.resample('10min').agg(res_aggs)

In [46]:
# Merge df and labels
df = pd.merge_asof(df, labels, left_index=True, right_index=True, tolerance=pd.Timedelta('10m'))

In [49]:
df[df.STRESSED.notnull()].shape

(76, 26)

In [ ]:
## HERE

In [112]:
df = merge_sms(df, sms)

In [ ]:
def get_sms(user, loc):
    sms = pd.read_csv(loc + 'sms/sms_' + user + '.csv', index_col=False)
    sms = sms[['timestamp']]
    sms = sms.groupby('timestamp').count().reset_index()
    sms['timestamp'] = pd.to_datetime(sms.timestamp, unit='s')
    sms['sms'] = 1
    sms = sms.set_index('timestamp')
    return sms

In [245]:
def get_app_usage(user, loc):
    app = pd.read_csv(loc + 'app_usage/running_app_' + user + '.csv', index_col=False)
    return app

In [247]:
app_usage = get_app_usage(user, dir_loc)

In [257]:
app_usage.RUNNING_TASKS_baseActivity_mPackage.unique()

array(['com.google.android.gm', 'com.android.launcher', 'com.rhmsoft.fm',
       'com.android.systemui', 'com.google.android.apps.paco',
       'com.google.android.googlequicksearchbox',
       'com.example.android.apis', 'com.android.settings',
       'com.android.phone', 'org.bewellapp',
       'com.android.packageinstaller', 'com.android.chrome',
       'com.google.android.gallery3d', 'com.google.android.deskclock',
       'com.android.contacts', 'edu.dartmouth.cs.camera',
       'edu.dartmouth.cs.myruns2', 'com.android.vending',
       'com.spotify.mobile.android.ui',
       'edu.dartmouth.cs.fragmentalertdialog', 'edu.dartmouth.cs.myruns1',
       'com.vmlite.vncserver', 'edu.dartmouth.cs.myruns',
       'com.google.android.calendar', 'edu.dartmouth.cs.hellocs65',
       'com.android.mms', 'edu.dartmouth.cs.myfirstapp',
       'edu.dartmouth.edu.hellocs65', 'edu.dartmouth.cs.helloworld',
       'com.dropbox.android', 'edu.dartmouth.edu.helloworld',
       'com.google.android.talk'

In [250]:
app_usage.head(10)

,id,device,timestamp,RUNNING_TASKS_baseActivity_mClass,RUNNING_TASKS_baseActivity_mPackage,RUNNING_TASKS_id,RUNNING_TASKS_numActivities,RUNNING_TASKS_numRunning,RUNNING_TASKS_topActivity_mClass,RUNNING_TASKS_topActivity_mPackage
0,a7d09e48-5bbd-4672-9f55-ea34bf4862ee-38,1977b545-a88f-4903-a7ae-2c434de4be49,1364100683,com.google.android.gm.ConversationListActivity...,com.google.android.gm,256,1,1,com.google.android.gm.ConversationListActivity...,com.google.android.gm
1,a7d09e48-5bbd-4672-9f55-ea34bf4862ee-38,1977b545-a88f-4903-a7ae-2c434de4be49,1364100683,com.android.launcher2.Launcher,com.android.launcher,2,1,1,com.android.launcher2.Launcher,com.android.launcher
2,a7d09e48-5bbd-4672-9f55-ea34bf4862ee-38,1977b545-a88f-4903-a7ae-2c434de4be49,1364100683,com.rhmsoft.fm.FileManager,com.rhmsoft.fm,265,1,1,com.rhmsoft.fm.FileManager,com.rhmsoft.fm
3,a7d09e48-5bbd-4672-9f55-ea34bf4862ee-38,1977b545-a88f-4903-a7ae-2c434de4be49,1364100683,com.android.systemui.recent.RecentsActivity,com.android.systemui,260,1,1,com.android.systemui.recent.RecentsActivity,com.android.systemui
4,a7d09e48-5bbd-4672-9f55-ea34bf4862ee-38,1977b545-a88f-4903-a7ae-2c434de4be49,1364100683,com.google.android.apps.paco.ExperimentManager...,com.google.android.apps.paco,262,1,1,com.google.android.apps.paco.ExperimentManager...,com.google.android.apps.paco
5,a7d09e48-5bbd-4672-9f55-ea34bf4862ee-38,1977b545-a88f-4903-a7ae-2c434de4be49,1364100683,com.google.android.googlequicksearchbox.Search...,com.google.android.googlequicksearchbox,263,1,1,com.google.android.googlequicksearchbox.Search...,com.google.android.googlequicksearchbox
6,a7d09e48-5bbd-4672-9f55-ea34bf4862ee-38,1977b545-a88f-4903-a7ae-2c434de4be49,1364100683,com.example.android.apis.ApiDemos,com.example.android.apis,259,3,3,com.example.android.apis.ApiDemos,com.example.android.apis
7,a7d09e48-5bbd-4672-9f55-ea34bf4862ee-38,1977b545-a88f-4903-a7ae-2c434de4be49,1364100683,com.android.settings.Settings$PowerUsageSummar...,com.android.settings,76,1,0,com.android.settings.Settings$PowerUsageSummar...,com.android.settings
8,a7d09e48-5bbd-4672-9f55-ea34bf4862ee-38,1977b545-a88f-4903-a7ae-2c434de4be49,1364100683,com.android.phone.InCallScreen,com.android.phone,61,1,1,com.android.phone.InCallScreen,com.android.phone
9,8b67cbb5-cfa2-465b-b5af-31ee2c6606a6-38,1977b545-a88f-4903-a7ae-2c434de4be49,1364099483,com.example.android.apis.ApiDemos,com.example.android.apis,259,3,3,com.example.android.apis.ApiDemos,com.example.android.apis


In [21]:
for user in user_codes:
    # Sensing
    activity = get_activity(user, dir_loc)
    print('activity data read for', user, 'is completed.')
    audio = get_audio(user, dir_loc)
    print('audio data read for', user, 'is completed.')
    conversation = get_conversation(user, dir_loc)
    print('conversation data read for', user, 'is completed.')
    bluetooth = get_bluetooth(user, dir_loc)
    print('bluetooth data read for', user, 'is completed.')
    wifi = get_wifi(user, dir_loc)
    print('wifi data read for', user, 'is completed.')
    dark = get_dark(user, dir_loc)
    print('dark data read for', user, 'is completed.')
    phone_charge = get_phone_charge(user, dir_loc)
    print('phone_charge data read for', user, 'is completed.')
    phone_lock = get_phone_lock(user, dir_loc)
    print('phone_lock data read for', user, 'is completed.')
    # Other Than Sensing
    sms = get_sms(user, dir_loc)
    print('sms data read for', user, 'is completed.')
    call_log = get_call_log(user, dir_loc)
    print('call_log data read for', user, 'is completed.')
    # EMA
    stress = ema(user, 'Stress', ['level'], dir_loc)
    print('stress data read for', user, 'is completed.')
    mood2 = ema(user, 'Mood 2', ['how'], dir_loc)
    print('mood2 data read for', user, 'is completed.')

    print('DATA READ IS COMPLETED.')

    df = merge_all(activity, audio, conversation,
                    bluetooth, wifi, dark,
                    phone_charge, phone_lock,
                   sms, call_log)

    print('Shape of df after merge:', str(df.shape))
    print('Data merge is completed.')

    # Create labels
    if stress.shape[1] == 2:
        stress['level'] = stress['level'].replace([1,2,3], 1)
        stress['level'] = stress['level'].replace([4,5], 0)
        stress.columns = ['STRESSED', 'resp_time']

    if mood2.shape[1] == 2:
        mood2 = mood2.replace([1, 3], 0)
        mood2 = mood2.replace([2], 1)
        mood2.columns = ['STRESSED', 'resp_time']

    labels = stress.append(mood2)
    labels = labels.sort_values(by='resp_time', ascending=True)
    labels = labels.set_index('resp_time')


    # Choose only valid timestamps
    df = df[df.timestamp.notnull()]

    # Fill empty values in dataset.
    df.loc[:, ['activity_inference',
               'audio_inference']] = df.loc[:, ['activity_inference',
                                                 'audio_inference']].fillna(value=3)

    df.loc[:, ['conversation',
                'phone_in_dark',
                'phone_charging',
                'phone_locked',
                'sms',
                'call_log']] = df.loc[:, ['conversation',
                                        'phone_in_dark',
                                        'phone_charging',
                                        'phone_locked',
                                        'sms',
                                        'call_log']].fillna(value=0)

    df.loc[:, ['activity_inference', 
               'audio_inference']] = df.loc[:, ['activity_inference', 
                                                'audio_inference']].astype(int)

    # One Hot Encode
    df['activity_inference'] = df['activity_inference'].astype('category')
    df['audio_inference'] = df['audio_inference'].astype('category')
    df = pd.get_dummies(df)

    # Resampling
    df = df.set_index('timestamp')
    df.index = pd.to_datetime(df.index, unit='s')

    res_aggs = resample_aggregations(list(df.columns))
    df = df.resample('10min').agg(res_aggs)

    # Merge df and labels
    df = pd.merge_asof(df, labels, left_index=True, right_index=True, tolerance=pd.Timedelta('10m'))

    df.to_csv('prepared_user_data/' + user + '_sensing_data.csv', index=True, header=True)

    print(user, 'is completed.')
    print('Shape of df is:', str(df.shape))

KeyboardInterrupt: 

In [ ]:
show_full_data(dataset.head(10))